In [1]:
%reload_ext autoreload
%autoreload 2

# Explain

In [2]:
from askai.explain import explain

In [3]:
from IPython.display import Markdown

def render_md(x):
    # if x.split('\n')[-1].strip() == '```':
    #     x = x.split('\n')[:-1]
    #     x.append('\```')
    #     x = '\n'.join(x)

    return Markdown('```\n' + x + '\n```')

In [4]:
import numpy as np

x = np.zeros((10, 10))
x

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
import pandas as pd

df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df

,a,b
0,1,4
1,2,5
2,3,6


In [6]:
render_md(explain(lambda x: x + 1))

```
object of type builtins.function
```

In [7]:
render_md(explain(x))

```
object of type numpy.ndarray
array of shape (10, 10) with dtype float64
```

In [8]:
render_md(explain(df))

```
object of type pandas.core.frame.DataFrame
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       3 non-null      int64
 1   b       3 non-null      int64
dtypes: int64(2)
memory usage: 176.0 bytes
```

# Examples

In [43]:
import inspect

def work_f():
    import numpy as np
    result = np
    return result

def inside_code(f):
    code = inspect.getsource(f).strip()
    code_lines = [l[4:] for l in code.split('\n')[1:-1]]
    in_code = '\n'.join(code_lines).strip()
    return in_code

render_md(inside_code(work_f))

```
import numpy as np
result = np
```

In [44]:
def work():
    import pandas as pd
    result = pd
    return result

package_example = {
    'task': 'that package for dataframes manipulation and stuff',
    'args': {},
    'code_f': work,
    'validate': True,
    'result': pd,
}

In [45]:
def work():
    result = 60 * 60 * 24 * 365 * 10
    return result

constant_example = {
    'task': 'number of seconds in 10 years',
    'args': {},
    'code_f': work,
    'validate': True,
    'result': 315_360_000,
}

In [46]:
def work_xy(x, y):
    result = x + y
    return result

def work_x(x):
    def sum(y):
        return x + y
    result = sum
    return result

def work_y(y):
    def sum(x):
        return x + y
    result = sum
    return result

def work():
    def sum(x, y):
        return x + y
    result = sum
    return result

args_examples = [
    {
        'task': 'sum of two numbers x and y',
        'args': {'x': 1, 'y': 2},
        'code_f': work_xy,
        'validate': True,
        'result': 3,
    },
    {
        'task': 'sum of two numbers x and y',
        'args': {'x': 1},
        'code_f': work_x,
        'validate': False,
    },
    {
        'task': 'sum of two numbers x and y',
        'args': {'y': 2},
        'code_f': work_y,
        'validate': False,
    },
    {
        'task': 'sum of two numbers x and y',
        'args': {},
        'code_f': work,
        'validate': False,
    }
]

In [47]:
def work_arg(filename):
    # filename is an passed arg so we use it
    with open(f'{filename}.txt') as f:
        content = f.read()
    result = content
    return result

def work_noarg():
    # filename isn't passed as arg but rather hardcoded
    with open(f'filename.txt') as f:
        content = f.read()
    result = content
    return result

files_examples = [
    {
        'task': 'read file filename.txt',
        'args': {'filename': 'pupupu'},
        'code_f': work_arg,
        'validate': True,
        'result': 'content',
    },
    {
        'task': 'read file filename.txt',
        'args': {},
        'code_f': work_noarg,
        'validate': True,
        'result': 'zzz',
    },
]

In [48]:
def work(a):
    import numpy as np
    import matplotlib.pyplot as plt

    # color isn't provided but mentioned so we made a func
    def plot_line(color):
        # generate x and y values
        # a is provided so we just reference it directly
        n = a.shape[0]
        x = np.linspace(0, n - 1, num=n)
        y = a

        # create a line plot using the specified color
        fig = plt.figure()
        plt.plot(x, y, color=color)

        # add known labels and title to the plot
        plt.ylabel('a')
        plt.title(f'line plot of a with {color} color')

        # user don't intend to observe outside effect
        plt.close(fig)

        return fig

    result = plot_line
    return result

plot_example = {
    'task': 'make line plot of `a` with specified color',
    'args': {'a': np.random.rand(10)},
    'code_f': work,
    'validate': False,
}

In [49]:
FEW_SHOT_EXAMPLES = [
    package_example,
    constant_example,
    *args_examples,
    *files_examples,
    plot_example,
]

for example in FEW_SHOT_EXAMPLES:
    result = example['code_f'](**example['args'])
    if example['validate']:
        assert result == example['result']

In [50]:
# from askai.prompt.prompt import make_example_prompt


# examples_lines = []
# for example in FEW_SHOT_EXAMPLES:
#     if len(examples_lines) > 0:
#         examples_lines.append('')
    
#     # multiple newlines inside
#     examples_lines.append(make_example_prompt(example['task'], **example['args']))

#     examples_lines.append('CODE:\n```python\n{}\n```'.format(inside_code(example['code_f'])))


# for line in examples_lines:
#     print(line)

In [51]:
# with open('examples.txt', 'w') as f:
#     f.write('\n'.join(examples_lines))

In [52]:
from askai.prompt.prompt import make_query

messages = []
for example in FEW_SHOT_EXAMPLES:
    query = make_query(example['task'], **example['args'])
    response = '```python\n{}\n```'.format(inside_code(example['code_f']))
    messages.append({'query': query, 'response': response})

messages

[{'query': 'TASK: """\nthat package for dataframes manipulation and stuff\n"""\nARGS: ',
  'response': '```python\nimport pandas as pd\nresult = pd\n```'},
 {'query': 'TASK: """\nnumber of seconds in 10 years\n"""\nARGS: ',
  'response': '```python\nresult = 60 * 60 * 24 * 365 * 10\n```'},
 {'query': 'TASK: """\nsum of two numbers x and y\n"""\nARGS: x, y\n- x: """\nobject of type builtins.int\n"""\n- y: """\nobject of type builtins.int\n"""',
  'response': '```python\nresult = x + y\n```'},
 {'query': 'TASK: """\nsum of two numbers x and y\n"""\nARGS: x\n- x: """\nobject of type builtins.int\n"""',
  'response': '```python\ndef sum(y):\n    return x + y\nresult = sum\n```'},
 {'query': 'TASK: """\nsum of two numbers x and y\n"""\nARGS: y\n- y: """\nobject of type builtins.int\n"""',
  'response': '```python\ndef sum(x):\n    return x + y\nresult = sum\n```'},
 {'query': 'TASK: """\nsum of two numbers x and y\n"""\nARGS: ',
  'response': '```python\ndef sum(x, y):\n    return x + y\nre

In [53]:
import json

with open('examples.json', 'w') as f:
    json.dump(messages, f)